In [0]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

E: Package 'python-software-properties' has no installation candidate
Selecting previously unselected package google-drive-ocamlfuse.
(Reading database ... 144467 files and directories currently installed.)
Preparing to unpack .../google-drive-ocamlfuse_0.7.22-0ubuntu1~ubuntu18.04.1_amd64.deb ...
Unpacking google-drive-ocamlfuse (0.7.22-0ubuntu1~ubuntu18.04.1) ...
Setting up google-drive-ocamlfuse (0.7.22-0ubuntu1~ubuntu18.04.1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope

In [12]:
!mkdir -p drive
!google-drive-ocamlfuse drive

/bin/bash: google-drive-ocamlfuse: command not found


In [0]:
import os
os.chdir("drive")
if("Colab Notebooks" in os.listdir()):
  os.chdir("Colab Notebooks")
else:
  os.mkdir("Colab Notebooks")
  os.chdir("Colab Notebooks")

In [0]:
!ls

In [0]:
# Keras深度學習模組套件
from keras.utils.np_utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.optimizers import RMSprop
from keras import Sequential
from keras import utils as np_utils
from keras import backend as K
from keras import optimizers
from keras.applications.resnet import ResNet50

# tensorflow深度學習模組套件
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow import keras
import tensorflow as tf

import numpy as np
import pandas as pd

In [11]:
train_label = pd.read_csv('AIMango/train.csv')
test_label = pd.read_csv('AIMango/dev.csv')
val_label = pd.read_csv('AIMango/dev.csv')

grade = {'A': '1','B': '2','C': '3'} 
train_label.label = [grade[item] for item in train_label.label]
val_label.label = [grade[item] for item in val_label.label]
test_label.label = [grade[item] for item in test_label.label]
print(train_label.label)

FileNotFoundError: ignored

In [0]:
datagen = ImageDataGenerator(
    rescale=1./255,
    zca_whitening=False,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

In [0]:
train_generator=datagen.flow_from_dataframe(dataframe=train_label, directory="C1-P1_Train", x_col="image_id", y_col="label", class_mode="categorical", target_size=(128,128), batch_size=8,validate_filenames=False)

val_datagen = ImageDataGenerator(rescale=1./255)
valid_generator=val_datagen.flow_from_dataframe(dataframe=val_label,directory="C1-P1_Dev", x_col="image_id", y_col="label", class_mode="categorical", target_size=(128,128), batch_size=8,validate_filenames=False)
test_datagen = ImageDataGenerator(rescale=1./255,horizontal_flip=True)
test_generator=test_datagen.flow_from_dataframe(dataframe=test_label,directory="C1-P1_Dev", x_col="image_id", y_col="label", class_mode="categorical", target_size=(128,128), batch_size=4,validate_filenames=False)

In [0]:
# 建立深度學習CNN Model

model = tf.keras.Sequential()

model.add(layers.Conv2D(16,(3,3),
                 strides=(1,1),
                 input_shape=(800, 800, 3),
                 padding='same',
                 activation='relu',
                 ))

model.add(layers.BatchNormalization())

model.add(layers.Conv2D(32,(3,3),
                 strides=(1,1),
                 input_shape=(800, 800, 3),
                 padding='same',
                 activation='relu',
                 ))
model.add(layers.Conv2D(32,(3,3),
                 strides=(2,2),
                 padding='same',
                 activation='relu',
                 ))
model.add(layers.Conv2D(32,(3,3),
                 strides=(2,2),
                 padding='same',
                 activation='relu',
                 ))
model.add(layers.Conv2D(32,(3,3),
                 strides=(5,5),
                 padding='same',
                 activation='relu',
                 ))
model.add(layers.Conv2D(64,(3,3),
                 strides=(1,1),
                 padding='same',
                 activation='relu',
                 ))
model.add(layers.Conv2D(32,(3,3),
                 strides=(5,5),
                 padding='same',
                 activation='relu',
                 ))
model.add(layers.Conv2D(32,(3,3),
                 strides=(2,2),
                 padding='same',
                 activation='relu',
                 ))
model.add(layers.Conv2D(32,(16,16),
                 strides=(2,2),
                 padding='same',
                 activation='relu',
                 ))
model.add(layers.Flatten())
model.add(layers.Dense(128,activation='relu'))
model.add(layers.Dropout(0.2))
model.add(layers.Dense(3,activation='softmax'))

model.summary()

In [0]:
from keras.optimizers import adam
adam = optimizers.adam(lr=2e-3)
model.compile(optimizer='adam',
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=['acc'])

In [0]:
# 檔名設定
file_name = 'e94076089_model'
# 加入EarlyStopping以及Tensorboard等回調函數
CB = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=2)
TB = keras.callbacks.TensorBoard(log_dir='./log'+"_"+file_name, histogram_freq=1)

In [0]:
# GPU Setting for keras
from tensorflow.compat.v1.keras.backend import set_session
 
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
config = tf.compat.v1.ConfigProto()
config.allow_soft_placement=True
config.gpu_options.per_process_gpu_memory_fraction = 0.8
config.gpu_options.allow_growth=True
sess = tf.compat.v1.Session(config = config)
set_session(sess)

In [0]:
STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=valid_generator.n//valid_generator.batch_size
model.fit_generator(generator=train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=valid_generator,  
                    validation_steps=STEP_SIZE_VALID,
                    callbacks=[CB],
                    epochs=2)

In [0]:
STEP_SIZE_TRAIN=train_generator.n //train_generator.batch_size
STEP_SIZE_VALID=valid_generator.n //valid_generator.batch_size
history = model.fit(
    x = train_generator , y = valid_generator,
    batch_size = train_generator.batch_size,
    epochs = STEP_SIZE_TRAIN,
    validation_split = 0.2,
    callbacks = [CB]
)

In [0]:
def plot_learning_curves(history):
    pd.DataFrame(history.history).plot(figsize=(8,5))
    plt.grid(True)
    plt.gca().set_ylim(0,1)
    plt.show()

In [0]:
plot_learning_curves(history)

In [0]:
# 測試集標籤預測
y_pred = model.predict(test_generator)

In [0]:
count = 0
for i in range(len(y_pred)):
    if(np.argmax(y_pred[i]) == np.argmax(y_test[i])): #argmax函数找到最大值的索引，即为其类别
        count += 1
score = count/len(y_pred)
print('正确率为:%.2f%s' % (score*100,'%'))